In [0]:
%pip install databricks_langchain
%restart_python

In [0]:
df = spark.table("workspace.car_sales.vehicles_enriched")

In [0]:
display(df)

In [0]:
df.columns

In [0]:
from langchain.agents import create_agent
from databricks_langchain import ChatDatabricks

In [0]:
import mlflow
mlflow.langchain.autolog()

In [0]:
system_prompt = """
You are the **AutoStrategist Supervisor**, an expert car sales consultant. Your goal is to assist users in selling their vehicles by determining an optimal listing price and generating a high-quality sales description.

You manage a team of two specialized sub-agents:
1.  **Market Analyst:** Accesses historical sales data to determine base market value.
2.  **Repair Specialist:** Accesses a database of component and labor costs to estimate repair deductions.

### YOUR RESPONSIBILITIES:

**1. Information Gathering (The Interview):**
You must collect specific details from the user to populate the required fields for your sub-agents. It may be that the user already provide you those information in his/her first message. Do not proceed to analysis until you have the "Critical" fields.
* **Critical Fields:** Manufacturer, Model, Year, Odometer (Mileage), Condition (Excellent, Good, Fair, Poor).
* **Secondary Fields (Ask if not provided, but optional):** Cylinders, Fuel Type, Transmission, Drive (FWD/RWD/AWD), Type (Sedan, SUV, etc.), Paint Color.
* **Defect Inquiry:** You must explicitly ask: "Are there any mechanical issues, warning lights, or cosmetic damage I should know about?"

**2. Orchestration & Delegation:**
* **If the user mentions damage or issues:** Call the **Repair Specialist** with the specific symptoms or components mentioned.
* **Once you have the vehicle specs:** Call the **Market Analyst** to get the historical average price and trends.

**3. Synthesis & Pricing Strategy:**
* Receive the *Base Market Value* from the Market Analyst.
* Receive the *Total Estimated Repair Costs* from the Repair Specialist (if any).
* **Calculate the Recommended List Price:** (Base Market Value) - (Repair Costs).
* *Strategy:* If the repair cost is low (<$300) and the value impact is high, advise the user to fix it before selling. If the cost is high, advise selling "as-is" with the price deduction.

**4. Final Output Generation:**
Once you have all data and agent reports, generate a final response containing:
* **The Assessment:** A breakdown of the market value, identified repair deductions, and the final suggested listing price range.
* **The "Seller's Copy":** A professional, compelling sales description ready for Craigslist/Facebook Marketplace. Highlight the car's features (from the secondary fields) and be honest but strategic about any "as-is" conditions.

### CONSTRAINTS & BEHAVIORS:
* **Be Proactive:** If the user says "I want to sell my Ford," do not call the agents yet. Ask: "I can help with that. What model and year is your Ford, and roughly how many miles are on it?"
* **Be Thorough:** If the user describes a noise (e.g., "squeaking when stopping"), ensure you ask the Repair Specialist about "brakes" or "pads" to get an accurate cost.
* **Tone:** Professional, encouraging, and data-driven.
* **Language:** Interact with the user in the language they initiate with, but ensure parameters passed to tools are standardized.

"""

prompt_market_analyst = """
You are the **Market Analyst**, an expert in analyzing car sales data. Your goal is to provide the user with the average market value of a car based on historical sales data.

### YOUR RESPONSIBILITIES:
* **Data Retrieval:** Access the historical sales data to find the average market value of a car based on the provided manufacturer, model, and

"""

market_analysit_description = """
Use this tool to determine the market value of a vehicle based on historical sales data. You must provide the manufacturer, model, and year.
If the odometer is provided, the tool will return weighted statistics for cars with similar mileage (+/- 20k miles). Returns a JSON object
containing the average price, median price, price standard deviation, and the number of similar cars sold
"""

repair_specialist_description = """
Use this tool to find the estimated cost of repairs. 
You must extract specific component names (like 'transmission', 'brake pads') from the user's text.

Inputs:
- diagnosis: (String) A brief description of the symptom (e.g., "squeaking noise").
- components: (List of Strings) A list of specific car parts to check. Even if there is only one part, provide a list. 
  Example: ["battery"] or ["brake pads", "rotors"].
"""

In [0]:
from pydantic import BaseModel, field_validator
from typing import Optional, List

class VehicleData(BaseModel):
    year: Optional[int] = None
    manufacturer: Optional[str] = None
    model: Optional[str] = None
    condition: Optional[str] = None
    cylinders: Optional[int] = None
    fuel: Optional[str] = None
    odometer: Optional[int] = None
    transmission: Optional[str] = None
    drive: Optional[str] = None
    car_type: Optional[str] = None
    paint_color: Optional[str] = None

class MarketAnalysisResults(BaseModel):
    average_price: Optional[float] = None
    median_price: Optional[float] = None
    price_std_dev: Optional[float] = None
    num_cars_sold: Optional[int] = None

class RepairData(BaseModel):
    diagnosis: Optional[str] = None
    components: Optional[List[str]] = None

    @field_validator('components', mode='before')
    def convert_string_to_list(cls, v):
        # If the LLM sends a string, wrap it in a list
        if isinstance(v, str):
            return [v]
        return v        

class RepairAnalysisResults(BaseModel):
    total_estimated_cost: Optional[float] = None
    identified_repairs: Optional[dict] = None


In [0]:
from langchain.tools import tool
from langchain.messages import HumanMessage, AIMessage
from langgraph.checkpoint.memory import MemorySaver

In [0]:
def get_table_schema_string(table_name: str) -> str:
    """
    Returns a string representation of the table schema 
    (Column Name, Type) to feed into the LLM context.
    """
    try:
        # Get the schema from Spark
        schema = spark.table(table_name).schema
        
        # Format it nicely for the LLM
        schema_str = f"Table: {table_name}\nColumns:\n"
        for field in schema:
            schema_str += f"- {field.name} ({field.dataType.simpleString()})\n"
            
        return schema_str
    except Exception as e:
        return f"Error fetching schema for {table_name}: {e}"
    
print(get_table_schema_string("workspace.car_sales.vehicles_enriched"))

In [0]:
from langchain_core.tools import tool
import json

@tool
def execute_market_sql(sql_query: str) -> str:
    """
    Executes a Spark SQL query against the car sales database.
    
    Args:
        sql_query: A valid Spark SQL query string. 
                   ALWAYS use the full table name: workspace.car_sales.vehicles_enriched
    """
    # 1. Safety Guardrails (Basic)
    forbidden_keywords = ["DROP", "DELETE", "ALTER", "TRUNCATE", "INSERT", "UPDATE"]
    if any(keyword in sql_query.upper() for keyword in forbidden_keywords):
        return "Error: Read-only access. Modification queries are not allowed."

    clean_query = sql_query.strip().rstrip(';')
    if "LIMIT" not in clean_query.upper():
        clean_query += " LIMIT 20"

    print(f"DEBUG: Executing SQL -> {clean_query}")

    try:
        df = spark.sql(clean_query)
        results = [row.asDict() for row in df.collect()]
        
        # --- NEW LOGIC START ---
        # If the query worked but found NO cars (count=0 or empty list)
        if not results or results[0].get('num_cars_sold', 0) == 0:
            return json.dumps({
                "found": False,
                "message": "Query returned 0 results. Try adding wildcards (e.g. ILIKE '%Mustang%') or removing the 'condition' filter."
            })
        # --- NEW LOGIC END ---

    except Exception as e:
        # Return the error so the LLM can self-correct!
        return f"SQL Execution Error: {str(e)}"

In [0]:
llm = ChatDatabricks(endpoint="databricks-gpt-oss-120b")

In [0]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

market_table_context = get_table_schema_string("workspace.car_sales.vehicles_enriched")

# Define the Prompt
market_analyst_system_prompt = f"""
You are an expert SQL Data Analyst for a car sales platform.
Your job is to query the database to answer questions about market trends, pricing, and inventory.

### DATABASE SCHEMA
You have access to the following table. You must ONLY query this table.
{market_table_context}

### GUIDELINES
1. **Always use the full table name** provided in the schema.
2. **Aggregation is Key:** Unless asked for a specific car, prefer calculating aggregations (AVG, MEDIAN, COUNT) to give market summaries.
3. **Fuzzy Matching:** - You are running on Databricks Spark SQL. 
   - You MUST use `ILIKE` (not LIKE) for all text matching to ensure case-insensitivity.
   - Do NOT use semicolons `;` at the end of your query.
4. **Dates:** The `year` column is an integer (e.g., 2015).
5. **Self-Correction:** If the tool returns an SQL error, analyze the error message, rewrite the query, and try again.

### CRITICAL RULES
1. **Always Use Wildcards:** Car model names in the database are messy (e.g., "Mustang GT", "F-150 XLT").
   - NEVER query `model = 'Mustang'`.
   - ALWAYS query `model ILIKE '%Mustang%'`.
2. **Relaxing Constraints:**
   - If a query returns `num_cars_sold: 0`, you MUST retry with fewer filters.
   - First retry: Remove `condition` and `paint_color`.
   - Second retry: Remove `year` (or use a range `year BETWEEN X AND Y`).
3. **Honesty Policy (Anti-Hallucination):**
   - If you cannot find data after 3 attempts, return a JSON with `null` values.
   - **DO NOT GUESS.** Do not use your internal knowledge to invent a price. If the tool says 0 cars, the answer is "Unknown".

### RETURN FORMAT
Your response should be a JSON object with the following keys:
- `average_price`: The average price of the cars in the dataset.
- `median_price`: The median price of the cars in the dataset.
- `price_std_dev`: The standard deviation of the prices in the dataset.
- `num_cars_sold`: The total number of cars sold in the dataset.

### ERROR HANDLING
If you are not able to get the resul, return a JSON with null values for all keys.

### EXAMPLE RESPONSE
```json
{{
    "average_price": 15000,
    "median_price": 12000,
    "price_std_dev": 2000,
    "num_cars_sold": 100
}}

### EXAMPLE RESPONSE ERROR
```json
{{
    "average_price": null,
    "median_price": null,
    "price_std_dev": null,
    "num_cars_sold": null
}}

```
"""

# Create the Agent
market_analyst_agent = create_agent(
    model=llm,
    tools=[execute_market_sql],
    system_prompt=market_analyst_system_prompt
)

In [0]:
# 1. Get Schema Context
repair_table_context = get_table_schema_string("workspace.car_sales.reparations")

# 2. Define the Prompt
repair_specialist_system_prompt = f"""
You are an expert Automotive Service Advisor.
Your job is to estimate repair costs by querying the database based on user descriptions of defects.

### DATABASE SCHEMA
You have access to the following table. You must ONLY query this table.
{repair_table_context}

### GUIDELINES
1. **Target Table:** Always query `workspace.car_sales.reparation_costs`.
2. **Search Strategy:** - Users use natural language (e.g., "weird noise", "leak"). 
   - You MUST use `ILIKE` on **BOTH** the `component` AND `diagnostic` columns.
   - Example: `WHERE lower(component) LIKE '%brake%' OR lower(diagnostic) LIKE '%brake%'`
3. **Multiple Issues:** If the user mentions multiple problems (e.g., "brakes and AC"), try to find rows matching ANY of those keywords.
4. **Self-Correction:** If the SQL fails, correct the syntax and retry.

### RETURN FORMAT
Your response should be a JSON object with the following keys:
- `identified_repairs`: A list of objects, where each object contains `component`, `diagnostic`, and `cost`.
- `total_estimated_cost`: The sum of all identified repair costs.

### EXAMPLE RESPONSE
```json
{{
    "identified_repairs": [
        {{ "component": "Brake Pads", "diagnostic": "squeaking noise", "cost": 250 }},
        {{ "component": "AC Compressor", "diagnostic": "blowing warm air", "cost": 900 }}
    ],
    "total_estimated_cost": 1150
}}

EXAMPLE RESPONSE (NO ISSUES FOUND)
{{
    "identified_repairs": null,
    "total_estimated_cost": null
}}

"""

# Create the Agent
repair_specialist_agent = create_agent(
    model=llm,
    tools=[execute_market_sql],
    system_prompt=repair_specialist_system_prompt
)

In [0]:
@tool("market_analyst", description=market_analysit_description)
def search_vehicle_database(vehicle_data: VehicleData) -> MarketAnalysisResults:
    """Search for price ranges for the given vehicle informtion.
    
    Args:
        vehicle_data (VehicleData): The vehicle data to search for.
        
    Returns:
        market_analysis_results (MarketAnalysisResults): The market analysis results
    """

    user_input = {"messages": [HumanMessage(content=vehicle_data.model_dump_json())]}
    res_search = market_analyst_agent.invoke(user_input)
    results_dict = json.loads(res_search['messages'][-1].content)
    market_analysis_results = results_dict # MarketAnalysisResults(**results_dict)

    return market_analysis_results


@tool("repair_specialist", description=repair_specialist_description)
def search_reparation_database(repair_data: RepairData) -> RepairAnalysisResults:
    """Search for price ranges for the given vehicle informtion.
    
    Args:
        repair_data (RepairData): The vehicle data to search for.
        
    Returns:
        repair_analysis_results (RepairAnalysisResults): The analysis of the reparation costs
    """

    user_input = {"messages": [HumanMessage(content=repair_data.model_dump_json())]}
    res_search = repair_specialist_agent.invoke(user_input)
    results_dict = json.loads(res_search['messages'][-1].content)
    repair_analysis_results = results_dict # RepairAnalysisResults(**results_dict)

    return repair_analysis_results

In [0]:
memory = MemorySaver()

graph = create_agent(
    model=llm,
    system_prompt=system_prompt,
    tools=[search_vehicle_database, search_reparation_database],
    checkpointer=memory
    )

In [0]:
def run_threaded_chat(graph):
    # Unique ID for this specific conversation session
    config = {"configurable": {"thread_id": "test_session_1"}}
    
    print("--- Starting Threaded Chat ---")
    
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["quit", "exit"]:
            break
            
        # With checkpointers, you ONLY pass the NEW message. 
        # The graph loads previous history automatically based on thread_id.
        events = graph.stream(
            {"messages": [HumanMessage(content=user_input)]},
            config=config,
            stream_mode="values"
        )
        
        # Print the output stream
        for event in events:
            if "messages" in event:
                last_msg = event["messages"][-1]
                # Only print if it's an AI message (avoid re-printing user input)
                if isinstance(last_msg, AIMessage):
                    print(f"Agent: {last_msg.content}")

run_threaded_chat(graph)

In [0]:
init_message = "I want to sell my Ford Mustang 2015 with 100000 miles."
result = graph.stream(init_message, config=config)

In [0]:
new_message = HumanMessage(content="I already provided you manufacturer, model, year and milage in the first message. Overall condition is good")
result = graph.invoke(new_message, config=config)

In [0]:
result